# 強弱盤

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np 
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import graphviz
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')
import pickle
from os import listdir
import os

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Mstat = pd.DataFrame()
for season in seasons:
    Mstat = pd.concat([Mstat, pd.read_csv(os.path.join(path, season, 'Miscellaneous_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Mstat.Team = Mstat.Team.map(Team_map)   
Mstat = Mstat.groupby('Team').mean().reset_index()
Mstat.head()

,Team,Rk,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Attend.,Attend./G
0,Atlanta Hawks,18.333333,26.416667,37.333333,42.166667,36.333333,43.166667,-1.875000,-0.133333,-2.010000,106.533333,108.250000,-1.716667,98.933333,0.256667,0.355000,0.552000,0.516000,13.950000,21.916667,0.197000,0.516500,13.700000,75.266667,0.201833,637171.166667,15997.333333
1,Boston Celtics,8.000000,25.300000,48.833333,31.500000,48.666667,31.666667,3.390000,-0.446667,2.941667,109.300000,105.866667,3.433333,97.700000,0.247833,0.348333,0.551667,0.514167,12.250000,23.000000,0.195333,0.500333,13.466667,76.283333,0.211000,728191.333333,18459.166667
2,Brooklyn Nets,22.000000,26.383333,30.666667,49.666667,31.000000,49.333333,-3.556667,-0.120000,-3.678333,106.183333,109.800000,-3.616667,98.383333,0.268833,0.344333,0.544667,0.507333,13.500000,22.600000,0.204167,0.515500,11.866667,76.116667,0.191833,621465.500000,15718.500000
3,Charlotte Hornets,18.333333,26.000000,35.833333,43.333333,37.000000,42.166667,-1.306667,-0.263333,-1.571667,107.183333,108.550000,-1.366667,96.216667,0.274833,0.333333,0.538667,0.497500,11.533333,21.633333,0.212833,0.520333,12.366667,78.483333,0.174333,659831.333333,16719.333333
4,Chicago Bulls,19.833333,26.016667,34.000000,45.166667,33.166667,46.000000,-2.763333,-0.245000,-3.008333,105.866667,108.616667,-2.750000,96.800000,0.247500,0.301667,0.534833,0.496667,12.800000,23.550000,0.192667,0.515667,12.500000,76.600000,0.190667,830800.666667,20798.500000


In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Tstat = pd.DataFrame()
for season in seasons:
    Tstat = pd.concat([Tstat, pd.read_csv(os.path.join(path, season, 'Team_Per_Game_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Tstat.Team = Tstat.Team.map(Team_map)   
Tstat = Tstat.groupby('Team').mean().reset_index()
Tstat.head()

,Team,Rk,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Atlanta Hawks,16.000000,79.500000,241.700000,39.166667,86.400000,0.453500,10.833333,30.800000,0.352667,28.333333,55.616667,0.509500,17.016667,22.183333,0.769333,9.650000,33.400000,43.050000,24.733333,8.366667,4.950000,15.633333,20.233333,106.166667
1,Boston Celtics,11.333333,80.333333,241.550000,39.733333,87.900000,0.452000,10.900000,30.583333,0.354500,28.850000,57.300000,0.504333,17.133333,21.766667,0.787167,10.283333,34.016667,44.300000,24.283333,8.200000,4.550000,13.566667,20.900000,107.533333
2,Brooklyn Nets,17.500000,80.333333,242.233333,38.716667,86.566667,0.447667,10.400000,29.983333,0.345500,28.333333,56.583333,0.502167,17.666667,23.283333,0.759167,10.150000,34.450000,44.600000,22.766667,6.833333,4.366667,15.116667,20.233333,105.500000
3,Charlotte Hornets,19.166667,79.166667,241.983333,37.783333,86.116667,0.438833,10.116667,28.750000,0.350500,27.666667,57.316667,0.483333,18.316667,23.666667,0.774167,9.800000,34.166667,43.950000,22.266667,6.833333,4.850000,12.583333,17.966667,104.050000
4,Chicago Bulls,23.166667,79.166667,242.050000,38.650000,87.116667,0.443667,9.283333,26.350000,0.353000,29.350000,60.800000,0.483667,16.750000,21.516667,0.777500,10.650000,33.966667,44.633333,22.616667,7.516667,4.700000,14.183333,19.333333,103.316667


In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Ostat = pd.DataFrame()
for season in seasons:
    Ostat = pd.concat([Ostat, pd.read_csv(os.path.join(path, season, 'Opponent_Per_Game_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Ostat.Team = Ostat.Team.map(Team_map)   
Ostat = Ostat.groupby('Team').mean().reset_index()
Ostat.head()

,Team,Rk,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Atlanta Hawks,17.333333,79.500000,241.700000,39.866667,87.400000,0.455833,10.650000,29.766667,0.355833,29.233333,57.633333,0.508000,17.716667,23.116667,0.765500,10.983333,34.266667,45.200000,24.483333,8.766667,5.416667,15.450000,20.500000,108.033333
1,Boston Celtics,10.333333,80.333333,241.550000,38.250000,85.950000,0.445000,9.483333,28.050000,0.337833,28.783333,57.916667,0.497500,18.150000,23.800000,0.762167,10.566667,34.316667,44.866667,22.133333,7.233333,4.966667,14.966667,19.916667,104.133333
2,Brooklyn Nets,22.500000,80.333333,242.233333,41.016667,89.183333,0.460167,9.916667,27.683333,0.358833,31.100000,61.483333,0.506167,17.133333,22.450000,0.761833,10.766667,34.866667,45.633333,23.200000,8.216667,5.233333,13.350000,20.266667,109.050000
3,Charlotte Hornets,12.666667,79.166667,241.983333,39.800000,86.616667,0.459000,10.700000,29.600000,0.360500,29.100000,57.000000,0.511833,15.116667,19.433333,0.776167,9.350000,35.416667,44.783333,24.516667,6.816667,5.383333,13.450000,20.783333,105.350000
4,Chicago Bulls,15.500000,79.166667,242.050000,39.883333,86.866667,0.459333,9.800000,27.633333,0.352167,30.100000,59.233333,0.511667,16.500000,21.716667,0.760833,10.400000,34.616667,45.016667,23.850000,7.583333,5.433333,13.783333,19.000000,106.100000


In [ ]:
# 統計每隊Miscellaneous Opponent，Team資料
def initialize_data(Mstat, Ostat, Tstat):
  new_Mstat = Mstat.drop(['Rk'], axis=1)
  new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
  new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)

  team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
  team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team')
  return team_stats1.set_index('Team', inplace=False, drop=True)

team_stats = initialize_data(Mstat, Ostat, Tstat)
team_stats.head(5)

,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Attend.,Attend./G,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,2P_x,2PA_x,2P%_x,FT_x,FTA_x,FT%_x,ORB_x,DRB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,PTS_x,FG_y,FGA_y,FG%_y,3P_y,3PA_y,3P%_y,2P_y,2PA_y,2P%_y,FT_y,FTA_y,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y
Team,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,26.416667,37.333333,42.166667,36.333333,43.166667,-1.875000,-0.133333,-2.010000,106.533333,108.250000,-1.716667,98.933333,0.256667,0.355000,0.552000,0.516000,13.950000,21.916667,0.197000,0.516500,13.700000,75.266667,0.201833,637171.166667,15997.333333,39.866667,87.400000,0.455833,10.650000,29.766667,0.355833,29.233333,57.633333,0.508000,17.716667,23.116667,0.765500,10.983333,34.266667,45.200000,24.483333,8.766667,5.416667,15.450000,20.500000,108.033333,39.166667,86.400000,0.453500,10.833333,30.800000,0.352667,28.333333,55.616667,0.509500,17.016667,22.183333,0.769333,9.650000,33.400000,43.050000,24.733333,8.366667,4.950000,15.633333,20.233333,106.166667
Boston Celtics,25.300000,48.833333,31.500000,48.666667,31.666667,3.390000,-0.446667,2.941667,109.300000,105.866667,3.433333,97.700000,0.247833,0.348333,0.551667,0.514167,12.250000,23.000000,0.195333,0.500333,13.466667,76.283333,0.211000,728191.333333,18459.166667,38.250000,85.950000,0.445000,9.483333,28.050000,0.337833,28.783333,57.916667,0.497500,18.150000,23.800000,0.762167,10.566667,34.316667,44.866667,22.133333,7.233333,4.966667,14.966667,19.916667,104.133333,39.733333,87.900000,0.452000,10.900000,30.583333,0.354500,28.850000,57.300000,0.504333,17.133333,21.766667,0.787167,10.283333,34.016667,44.300000,24.283333,8.200000,4.550000,13.566667,20.900000,107.533333
Brooklyn Nets,26.383333,30.666667,49.666667,31.000000,49.333333,-3.556667,-0.120000,-3.678333,106.183333,109.800000,-3.616667,98.383333,0.268833,0.344333,0.544667,0.507333,13.500000,22.600000,0.204167,0.515500,11.866667,76.116667,0.191833,621465.500000,15718.500000,41.016667,89.183333,0.460167,9.916667,27.683333,0.358833,31.100000,61.483333,0.506167,17.133333,22.450000,0.761833,10.766667,34.866667,45.633333,23.200000,8.216667,5.233333,13.350000,20.266667,109.050000,38.716667,86.566667,0.447667,10.400000,29.983333,0.345500,28.333333,56.583333,0.502167,17.666667,23.283333,0.759167,10.150000,34.450000,44.600000,22.766667,6.833333,4.366667,15.116667,20.233333,105.500000
Charlotte Hornets,26.000000,35.833333,43.333333,37.000000,42.166667,-1.306667,-0.263333,-1.571667,107.183333,108.550000,-1.366667,96.216667,0.274833,0.333333,0.538667,0.497500,11.533333,21.633333,0.212833,0.520333,12.366667,78.483333,0.174333,659831.333333,16719.333333,39.800000,86.616667,0.459000,10.700000,29.600000,0.360500,29.100000,57.000000,0.511833,15.116667,19.433333,0.776167,9.350000,35.416667,44.783333,24.516667,6.816667,5.383333,13.450000,20.783333,105.350000,37.783333,86.116667,0.438833,10.116667,28.750000,0.350500,27.666667,57.316667,0.483333,18.316667,23.666667,0.774167,9.800000,34.166667,43.950000,22.266667,6.833333,4.850000,12.583333,17.966667,104.050000
Chicago Bulls,26.016667,34.000000,45.166667,33.166667,46.000000,-2.763333,-0.245000,-3.008333,105.866667,108.616667,-2.750000,96.800000,0.247500,0.301667,0.534833,0.496667,12.800000,23.550000,0.192667,0.515667,12.500000,76.600000,0.190667,830800.666667,20798.500000,39.883333,86.866667,0.459333,9.800000,27.633333,0.352167,30.100000,59.233333,0.511667,16.500000,21.716667,0.760833,10.400000,34.616667,45.016667,23.850000,7.583333,5.433333,13.783333,19.000000,106.100000,38.650000,87.116667,0.443667,9.283333,26.350000,0.353000,29.350000,60.800000,0.483667,16.750000,21.516667,0.777500,10.650000,33.966667,44.633333,22.616667,7.516667,4.700000,14.183333,19.333333,103.316667


In [ ]:
# 計算ELO值，分辨隊伍強隊
def get_elo(team):
  try:
    return team_elos[team]
  except:
    team_elos[team] = base_elo
    return team_elos[team]

def calc_elo(win_team, lose_team):
  winner_rank = get_elo(win_team)
  loser_rank = get_elo(lose_team)

  rank_diff = winner_rank - loser_rank
  exp = (rank_diff  * -1) / 400
  odds = 1 / (1 + math.pow(10, exp))
  if winner_rank < 2100:
    k = 32
  elif winner_rank >= 2100 and winner_rank < 2400:
    k = 24
  else:
    k = 16
  new_winner_rank = round(winner_rank + (k * (1 - odds)))
  new_rank_diff = new_winner_rank - winner_rank
  new_loser_rank = loser_rank - new_rank_diff

  return new_winner_rank, new_loser_rank

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Games = pd.DataFrame()
for season in seasons:
    Games = pd.concat([Games, pd.read_csv(os.path.join(path, season, 'result.csv'))], ignore_index=True)
FTR = []
  
for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    if row['PTS']>row['PTS.1']:
        FTR.append('Visitor')
    else:
        FTR.append('Home')
Games["FTR"] =   FTR  
Games = Games[['Visitor/Neutral','Home/Neutral','FTR']]
Games

,Visitor/Neutral,Home/Neutral,FTR
0,Orlando Magic,New Orleans Pelicans,Home
1,Dallas Mavericks,San Antonio Spurs,Home
2,Houston Rockets,Los Angeles Lakers,Visitor
3,Milwaukee Bucks,Charlotte Hornets,Home
4,Philadelphia 76ers,Indiana Pacers,Home
...,...,...,...
7506,Los Angeles Clippers,Golden State Warriors,Visitor
7507,Detroit Pistons,Philadelphia 76ers,Home
7508,New York Knicks,Atlanta Hawks,Visitor
7509,Charlotte Hornets,Miami Heat,Visitor


In [ ]:
import random,math

base_elo = 1600
team_elos = {} 
x = []
y = []

def  build_dataSet(all_data):

  print("Building data set..")
  skip = 0
  for index, row in all_data.iterrows():

    Visitor = row['Visitor/Neutral']
    Home = row['Home/Neutral']
    FTR = row['FTR']

    Home_elo = get_elo(Home)
    Visitor_elo = get_elo(Visitor)

    #给主场比赛的队伍加上100的elo值
    Home_elo += 100

    Home_features = [Home_elo]
    Visitor_features = [Visitor_elo]
 
    for key, value in team_stats.loc[Home].iteritems():
      Home_features.append(value)
    for key, value in team_stats.loc[Visitor].iteritems():
      Visitor_features.append(value)

    x.append(Home_features+Visitor_features)
    if row['FTR'] == 'Home':
        y.append(0)
    else:
        y.append(1)

    if row['FTR'] == 'Home':
        new_winner_rank, new_loser_rank = calc_elo(Home, Visitor)
        team_elos[Home] = new_winner_rank
        team_elos[Visitor] = new_loser_rank
    else:
        new_winner_rank, new_loser_rank = calc_elo(Visitor, Home)
        team_elos[Visitor] = new_winner_rank
        team_elos[Home] = new_loser_rank

  return np.nan_to_num(x), y

x, y = build_dataSet(Games)

Building data set..


In [ ]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score

print("Fitting on %d game samples.." % len(x))
model = LogisticRegression()
model.fit(x, y)

print("Doing cross-validation..")
print(cross_val_score(model, x, y, cv = 10, scoring='accuracy', n_jobs=-1).mean())

Fitting on 7511 game samples..
Doing cross-validation..
0.6600978836728333


In [ ]:
import pickle #pickle模組

with open('強弱盤.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
#讀取Model
with open('強弱盤.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
def predict_winner(home, visitor, model):
  features = []

  features.append(get_elo(home) + 100)
  for key, value in team_stats.loc[home].iteritems():
    features.append(value)

  features.append(get_elo(visitor))
  for key, value in team_stats.loc[visitor].iteritems():
    features.append(value)


  features = np.nan_to_num(features)
  return model.predict_proba([features])

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[-2:-1]
Games = pd.DataFrame()
for season in seasons:
    Games = pd.concat([Games, pd.read_csv(os.path.join(path, season, 'result.csv'))], ignore_index=True)

FTR = []
for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    if row['PTS']>row['PTS.1']:
        FTR.append('Visitor')
    else:
        FTR.append('Home')
Games["FTR"] =  FTR  


Games = Games[['Visitor/Neutral','Home/Neutral','FTR']]
print('Predicting on new schedule..')
result = []
for index, row in Games.iterrows():
  Visitor = row['Visitor/Neutral']
  Home = row['Home/Neutral']
  pred = predict_winner(Home, Visitor, model)
  if pred[0][0] > pred[0][1]:
      result.append('Home')
  else:
      result.append('Visitor')

Predicting on new schedule..


In [ ]:
import numpy as np
correct = 0
for i in range(len(result)):
    if result[i] == Games['FTR'][i]:
        correct+=1
print('accuracy:',correct/len(result))

accuracy: 0.5447154471544715


# 大小盤

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Mstat = pd.DataFrame()
for season in seasons:
    Mstat = pd.concat([Mstat, pd.read_csv(os.path.join(path, season, 'Miscellaneous_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Mstat.Team = Mstat.Team.map(Team_map)   
Mstat = Mstat.groupby('Team').mean().reset_index()


In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Tstat = pd.DataFrame()
for season in seasons:
    Tstat = pd.concat([Tstat, pd.read_csv(os.path.join(path, season, 'Team_Per_Game_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Tstat.Team = Tstat.Team.map(Team_map)   
Tstat = Tstat.groupby('Team').mean().reset_index()


In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Ostat = pd.DataFrame()
for season in seasons:
    Ostat = pd.concat([Ostat, pd.read_csv(os.path.join(path, season, 'Opponent_Per_Game_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Ostat.Team = Ostat.Team.map(Team_map)   
Ostat = Ostat.groupby('Team').mean().reset_index()


In [ ]:
# 統計每隊Miscellaneous Opponent，Team資料
def initialize_data(Mstat, Ostat, Tstat):
  new_Mstat = Mstat.drop(['Rk'], axis=1)
  new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
  new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)

  team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
  team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team')
  return team_stats1.set_index('Team', inplace=False, drop=True)

team_stats = initialize_data(Mstat, Ostat, Tstat)


In [ ]:
# 計算ELO值，分辨隊伍強隊
def get_elo(team):
  try:
    return team_elos[team]
  except:
    team_elos[team] = base_elo
    return team_elos[team]

def calc_elo(win_team, lose_team):
  winner_rank = get_elo(win_team)
  loser_rank = get_elo(lose_team)

  rank_diff = winner_rank - loser_rank
  exp = (rank_diff  * -1) / 400
  odds = 1 / (1 + math.pow(10, exp))
  if winner_rank < 2100:
    k = 32
  elif winner_rank >= 2100 and winner_rank < 2400:
    k = 24
  else:
    k = 16
  new_winner_rank = round(winner_rank + (k * (1 - odds)))
  new_rank_diff = new_winner_rank - winner_rank
  new_loser_rank = loser_rank - new_rank_diff

  return new_winner_rank, new_loser_rank

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-1]
Games = pd.DataFrame()
for season in seasons:
    Games = pd.concat([Games, pd.read_csv(os.path.join(path, season, 'result.csv'))], ignore_index=True)
SUM = []

for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    SUM.append(row['PTS']+row['PTS.1'])
Games["SUM"] =   SUM 

FTR = []
for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    if row['PTS']>row['PTS.1']:
        FTR.append('Visitor')
    else:
        FTR.append('Home')
Games["FTR"] =  FTR  
 
Games = Games[['Visitor/Neutral','Home/Neutral','SUM',"FTR"]]

In [ ]:
import random,math

base_elo = 1600
team_elos = {} 
x = []
y = []

def  build_dataSet(all_data):

  print("Building data set..")
  skip = 0
  for index, row in all_data.iterrows():

    Visitor = row['Visitor/Neutral']
    Home = row['Home/Neutral']
    SUM = row['SUM']

    Home_elo = get_elo(Home)
    Visitor_elo = get_elo(Visitor)

    #给主场比赛的队伍加上100的elo值
    Home_elo += 100

    Home_features = [Home_elo]
    Visitor_features = [Visitor_elo]
 
    for key, value in team_stats.loc[Home].iteritems():
      Home_features.append(value)
    for key, value in team_stats.loc[Visitor].iteritems():
      Visitor_features.append(value)

    x.append(Home_features+Visitor_features)

    y.append(SUM)
 

    if row['FTR'] == 'Home':
        new_winner_rank, new_loser_rank = calc_elo(Home, Visitor)
        team_elos[Home] = new_winner_rank
        team_elos[Visitor] = new_loser_rank
    else:
        new_winner_rank, new_loser_rank = calc_elo(Visitor, Home)
        team_elos[Visitor] = new_winner_rank
        team_elos[Home] = new_loser_rank



  return np.nan_to_num(x), y

x, y = build_dataSet(Games)

Building data set..


In [ ]:
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score

print("Fitting on %d game samples.." % len(x))

model = LinearRegression()
model.fit(x, y)

Fitting on 8987 game samples..


LinearRegression()

In [ ]:
import pickle #pickle模組

with open('大小盤.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
#讀取Model
with open('大小盤.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
def predict_winner(home, visitor, model):
  features = []

  features.append(get_elo(home) + 100)
  for key, value in team_stats.loc[home].iteritems():
    features.append(value)

  features.append(get_elo(visitor))
  for key, value in team_stats.loc[visitor].iteritems():
    features.append(value)


  features = np.nan_to_num(features)
  return model.predict([features])

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[-2:-1]
Games = pd.DataFrame()
for season in seasons:
    Games = pd.concat([Games, pd.read_csv(os.path.join(path, season, 'result.csv'))], ignore_index=True)

FTR = []
SUM = []
for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    if row['PTS']>row['PTS.1']:
        FTR.append('Visitor')
    else:
        FTR.append('Home')
    SUM.append(row['PTS']+row['PTS.1'])
    

Games["FTR"] =  FTR  
Games["SUM"] =  SUM 

Games = Games[['Visitor/Neutral','Home/Neutral','FTR','SUM']]
print('Predicting on new schedule..')
result = []
for index, row in Games.iterrows():
  Visitor = row['Visitor/Neutral']
  Home = row['Home/Neutral']
  pred = predict_winner(Home, Visitor, model)
  result.append(pred)


Predicting on new schedule..


In [ ]:
np.array(Games.SUM).shape

(1476,)

In [ ]:
np.array(result).reshape(len(result)).shape

(1476,)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(np.array(Games.SUM),np.array(result).reshape(len(result)))

473.87355009370754

# 讓分盤

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Mstat = pd.DataFrame()
for season in seasons:
    Mstat = pd.concat([Mstat, pd.read_csv(os.path.join(path, season, 'Miscellaneous_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Mstat.Team = Mstat.Team.map(Team_map)   
Mstat = Mstat.groupby('Team').mean().reset_index()

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Tstat = pd.DataFrame()
for season in seasons:
    Tstat = pd.concat([Tstat, pd.read_csv(os.path.join(path, season, 'Team_Per_Game_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Tstat.Team = Tstat.Team.map(Team_map)   
Tstat = Tstat.groupby('Team').mean().reset_index()

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-2]
Ostat = pd.DataFrame()
for season in seasons:
    Ostat = pd.concat([Ostat, pd.read_csv(os.path.join(path, season, 'Opponent_Per_Game_Stat.csv'))], ignore_index=True)

def Team_map(team):
    return team if team.find('*') == -1 else team[:-1]

Ostat.Team = Ostat.Team.map(Team_map)   
Ostat = Ostat.groupby('Team').mean().reset_index()

In [ ]:
# 統計每隊Miscellaneous Opponent，Team資料
def initialize_data(Mstat, Ostat, Tstat):
  new_Mstat = Mstat.drop(['Rk'], axis=1)
  new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
  new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)

  team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
  team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team')
  return team_stats1.set_index('Team', inplace=False, drop=True)

team_stats = initialize_data(Mstat, Ostat, Tstat)

In [ ]:
# 計算ELO值，分辨隊伍強隊
def get_elo(team):
  try:
    return team_elos[team]
  except:
    team_elos[team] = base_elo
    return team_elos[team]

def calc_elo(win_team, lose_team):
  winner_rank = get_elo(win_team)
  loser_rank = get_elo(lose_team)

  rank_diff = winner_rank - loser_rank
  exp = (rank_diff  * -1) / 400
  odds = 1 / (1 + math.pow(10, exp))
  if winner_rank < 2100:
    k = 32
  elif winner_rank >= 2100 and winner_rank < 2400:
    k = 24
  else:
    k = 16
  new_winner_rank = round(winner_rank + (k * (1 - odds)))
  new_rank_diff = new_winner_rank - winner_rank
  new_loser_rank = loser_rank - new_rank_diff

  return new_winner_rank, new_loser_rank

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[0:-1]
Games = pd.DataFrame()
for season in seasons:
    Games = pd.concat([Games, pd.read_csv(os.path.join(path, season, 'result.csv'))], ignore_index=True)
H_score = []
A_score = []

for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    H_score.append(row['PTS.1'])
    A_score.append(row['PTS'])

Games["H_score"] =   H_score 
Games["A_score"] =   A_score 

FTR = []
for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    if row['PTS']>row['PTS.1']:
        FTR.append('Visitor')
    else:
        FTR.append('Home')
Games["FTR"] =  FTR  
 
Games = Games[['Visitor/Neutral','Home/Neutral','A_score','H_score',"FTR"]]
Games

,Visitor/Neutral,Home/Neutral,A_score,H_score,FTR
0,Orlando Magic,New Orleans Pelicans,84,101,Home
1,Dallas Mavericks,San Antonio Spurs,100,101,Home
2,Houston Rockets,Los Angeles Lakers,108,90,Visitor
3,Milwaukee Bucks,Charlotte Hornets,106,108,Home
4,Philadelphia 76ers,Indiana Pacers,91,103,Home
...,...,...,...,...,...
8982,Indiana Pacers,Minnesota Timberwolves,98,100,Home
8983,Cleveland Cavaliers,Dallas Mavericks,114,96,Visitor
8984,Washington Wizards,San Antonio Spurs,99,116,Home
8985,Portland Trail Blazers,Utah Jazz,107,129,Home


In [ ]:
import random,math

base_elo = 1600
team_elos = {} 
x = []
y = []

def  build_dataSet(all_data):

  print("Building data set..")
  skip = 0
  for index, row in all_data.iterrows():

    Visitor = row['Visitor/Neutral']
    Home = row['Home/Neutral']
    H_score = row['H_score']
    A_score = row['A_score']

    Home_elo = get_elo(Home)
    Visitor_elo = get_elo(Visitor)

    #给主场比赛的队伍加上100的elo值
    Home_elo += 100

    Home_features = [Home_elo]
    Visitor_features = [Visitor_elo]
 
    for key, value in team_stats.loc[Home].iteritems():
      Home_features.append(value)
    for key, value in team_stats.loc[Visitor].iteritems():
      Visitor_features.append(value)

    x.append(Home_features+Visitor_features)

    y.append([H_score,A_score])
 

    if row['FTR'] == 'Home':
        new_winner_rank, new_loser_rank = calc_elo(Home, Visitor)
        team_elos[Home] = new_winner_rank
        team_elos[Visitor] = new_loser_rank
    else:
        new_winner_rank, new_loser_rank = calc_elo(Visitor, Home)
        team_elos[Visitor] = new_winner_rank
        team_elos[Home] = new_loser_rank



  return np.nan_to_num(x), y

x, y = build_dataSet(Games)

Building data set..


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score

print("Fitting on %d game samples.." % len(x))

model = RandomForestRegressor()
model.fit(x, y)

Fitting on 8987 game samples..


RandomForestRegressor()

In [ ]:
import pickle #pickle模組

with open('讓分盤.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
#讀取Model
with open('讓分盤.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
def predict_winner(home, visitor, model):
  features = []

  features.append(get_elo(home) + 100)
  for key, value in team_stats.loc[home].iteritems():
    features.append(value)

  features.append(get_elo(visitor))
  for key, value in team_stats.loc[visitor].iteritems():
    features.append(value)


  features = np.nan_to_num(features)
  return model.predict([features])[0]

In [ ]:
path = './NBA_DATA'
seasons = sorted(listdir(path))[-2:-1]
Games = pd.DataFrame()
for season in seasons:
    Games = pd.concat([Games, pd.read_csv(os.path.join(path, season, 'result.csv'))], ignore_index=True)
Games

H_score = []
A_score = []

for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
    H_score.append(row['PTS.1'])
    A_score.append(row['PTS'])

Games["H_score"] =   H_score 
Games["A_score"] =   A_score 

Games = Games[['Visitor/Neutral','Home/Neutral','H_score','A_score']]
print('Predicting on new schedule..')
result = []
for index, row in Games.iterrows():
  Visitor = row['Visitor/Neutral']
  Home = row['Home/Neutral']
  pred = predict_winner(Home, Visitor, model)
  result.append(pred)

Predicting on new schedule..


In [ ]:
result

# 本機運行

In [ ]:
import pickle
import pandas as pd
from datetime import datetime
from datetime import datetime,timezone,timedelta

 
class NBAPredict(object):
    def __init__(self):
        with open('強弱盤.pickle', 'rb') as f:
           self.StrongWeakModel = pickle.load(f)
        with open('大小盤.pickle', 'rb') as f:
           self.BigSmallModel =  pickle.load(f)
        with open('讓分盤.pickle', 'rb') as f:
           self.Handicap = pickle.load(f)

    def FetchDateTimeRange(self, day = 1):
        return (datetime.now().astimezone(timezone(timedelta(hours=8)))+timedelta(days=1)).replace(hour=23, minute=59,second=59).strftime("%Y-%m-%d %H:%M:%S.000")
    
    def process_DataTime(self,Date,Start):
        return datetime.strptime(Date+' '+Start[:-1],"%a, %b %d, %Y %I:%M")

    def FetchMatchEntrys(self,mydatetime):
        games = pd.read_csv('./NBA_DATA/明日賽事.csv')
        for index,game in games.iteritems():
            print(game)
  

    def predict_Handicap(self,home, visitor):
        features = []

        features.append(get_elo(home) + 100)
        for key, value in self.team_stats.loc[home].iteritems():
          features.append(value)

        features.append(get_elo(visitor))
        for key, value in self.team_stats.loc[visitor].iteritems():
          features.append(value)


        features = np.nan_to_num(features)
        return features#self.Handicap.predict([features])[0]

    def predict_SUM(self,home, visitor):
        features = []

        features.append(get_elo(home) + 100)
        for key, value in self.team_stats.loc[home].iteritems():
          features.append(value)

        features.append(get_elo(visitor))
        for key, value in self.team_stats.loc[visitor].iteritems():
          features.append(value)


        features = np.nan_to_num(features)
        return self.BigSmallModel.predict([features])[0]
    
    
    def predict_StrongWeak(self, home, visitor):
        features = []
        teams = [home, visitor]

        features.append(get_elo(home) + 100)
        for key, value in self.team_stats.loc[home].iteritems():
            features.append(value)

        features.append(get_elo(visitor))
        for key, value in self.team_stats.loc[visitor].iteritems():
            features.append(value)

        features = np.nan_to_num(features)

        return teams[self.StrongWeakModel.predict([features])[0]]
    
    def initialize(self):
        path = './NBA_DATA'
        seasons = sorted(listdir(path))[0:-2]
        Mstat = pd.DataFrame()
        for season in seasons:
           Mstat = pd.concat([Mstat, pd.read_csv(os.path.join(path, season, 'Miscellaneous_Stat.csv'))], ignore_index=True)
        def Team_map(team):
           return team if team.find('*') == -1 else team[:-1]
        Mstat.Team = Mstat.Team.map(Team_map)   
        Mstat = Mstat.groupby('Team').mean().reset_index()

        path = './NBA_DATA'
        seasons = sorted(listdir(path))[0:-2]
        Tstat = pd.DataFrame()
        for season in seasons:
          Tstat = pd.concat([Tstat, pd.read_csv(os.path.join(path, season, 'Team_Per_Game_Stat.csv'))], ignore_index=True)

        def Team_map(team):
          return team if team.find('*') == -1 else team[:-1]

        Tstat.Team = Tstat.Team.map(Team_map)   
        Tstat = Tstat.groupby('Team').mean().reset_index()

        path = './NBA_DATA'
        seasons = sorted(listdir(path))[0:-2]
        Ostat = pd.DataFrame()
        for season in seasons:
           Ostat = pd.concat([Ostat, pd.read_csv(os.path.join(path, season, 'Opponent_Per_Game_Stat.csv'))], ignore_index=True)

        def Team_map(team):
           return team if team.find('*') == -1 else team[:-1]

        Ostat.Team = Ostat.Team.map(Team_map)   
        Ostat = Ostat.groupby('Team').mean().reset_index()

        def initialize_data(Mstat, Ostat, Tstat):
           new_Mstat = Mstat.drop(['Rk'], axis=1)
           new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
           new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)

           team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
           team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team')
           return team_stats1.set_index('Team', inplace=False, drop=True)

        self.team_stats = initialize_data(Mstat, Ostat, Tstat)

        # 計算ELO值，分辨隊伍強隊
        def get_elo(team):
             try:
               
               return self.team_elos[team]
             except:
               self.team_elos[team] = base_elo
               return self.team_elos[team]

        def calc_elo(win_team, lose_team):
               winner_rank = get_elo(win_team)
               loser_rank = get_elo(lose_team)

               rank_diff = winner_rank - loser_rank
               exp = (rank_diff  * -1) / 400
               odds = 1 / (1 + math.pow(10, exp))
               if winner_rank < 2100:
                   k = 32
               elif winner_rank >= 2100 and winner_rank < 2400:
                   k = 24
               else:
                   k = 16
               new_winner_rank = round(winner_rank + (k * (1 - odds)))
               new_rank_diff = new_winner_rank - winner_rank
               new_loser_rank = loser_rank - new_rank_diff

               return new_winner_rank, new_loser_rank

        path = './NBA_DATA'
        seasons = sorted(listdir(path))[0:-1]
        Games = pd.DataFrame()
        for season in seasons:
          Games = pd.concat([Games, pd.read_csv(os.path.join(path, season, 'result.csv'))], ignore_index=True)
        
        H_score = []
        A_score = []
        SUM = []
        for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
            H_score.append(row['PTS.1'])
            A_score.append(row['PTS'])
            SUM.append(row['PTS.1']+row['PTS'])

        Games["H_score"] =   H_score 
        Games["A_score"] =   A_score
        Games["SUM"] =   SUM
        
        FTR = []
        for index, row in Games[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']].iterrows():
           if row['PTS']>row['PTS.1']:
                 FTR.append('Visitor')
           else:
                 FTR.append('Home')

        Games["FTR"] =  FTR  
        Games = Games[['Visitor/Neutral','Home/Neutral','A_score','H_score','SUM',"FTR"]]


        import random,math

        base_elo = 1600
        self.team_elos = {} 
        x = []
        y = []

        def  build_dataSet(all_data):
      
         print("Building data set..")
         skip = 0
         for index, row in all_data.iterrows():
     
           Visitor = row['Visitor/Neutral']
           Home = row['Home/Neutral']
           H_score = row['H_score']
           A_score = row['A_score']

           Home_elo = get_elo(Home)
           Visitor_elo = get_elo(Visitor)

           #给主场比赛的队伍加上100的elo值
           Home_elo += 100

           Home_features = [Home_elo]
           Visitor_features = [Visitor_elo]
 
           for key, value in team_stats.loc[Home].iteritems():
              Home_features.append(value)
           for key, value in team_stats.loc[Visitor].iteritems():
              Visitor_features.append(value)

           x.append(Home_features+Visitor_features)

           y.append([H_score,A_score])
 

           if row['FTR'] == 'Home':
              new_winner_rank, new_loser_rank = calc_elo(Home, Visitor)
              self.team_elos[Home] = new_winner_rank
              self.team_elos[Visitor] = new_loser_rank
           else:
              new_winner_rank, new_loser_rank = calc_elo(Visitor, Home)
              self.team_elos[Visitor] = new_winner_rank
              self.team_elos[Home] = new_loser_rank
         return np.nan_to_num(x), y
        build_dataSet(Games)      
        


a = NBAPredict()

a.initialize()
print(a.predict_StrongWeak('Orlando Magic','New Orleans Pelicans'))
print(a.predict_SUM('Orlando Magic','New Orleans Pelicans'))
print(len(a.predict_Handicap('Orlando Magic','New Orleans Pelicans')))

Building data set..
New Orleans Pelicans
212.08181762695312
136


In [ ]:
from datetime import datetime
from datetime import datetime,timezone,timedelta

(datetime.now().astimezone(timezone(timedelta(hours=8)))+timedelta(days=1)).replace(hour=23, minute=59,second=59).strftime("%a, %b %d, %Y %I:%M")

'Thu, Dec 02, 2021 11:59'